# Imports

In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import models,transforms,datasets
import torch
import bcolz
import time
%matplotlib inline

In [ ]:
# !pip install bcolz

In [1]:
import imp
import utils; imp.reload(utils)
from utils import *

GPU test

In [3]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

Using gpu: True 


# Data process

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
# data_dir = 'D:\\Tu Beo\\Education\\FoodVisor\\data\\UPMC_Food101\\images'
data_dir = '/home/foodlovers/FoodVisor/data/images'

In [6]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'test']}

In [6]:
len(dsets['train'].classes)

101

In [ ]:
dsets['train'].class_to_idx

In [ ]:
dsets['train'].imgs[:5]

In [7]:
dset_classes = dsets['train'].classes

In [7]:
dset_sizes = {x: len(dsets[x]) for x in ['train', 'test']}
dset_sizes

{'train': 67988, 'test': 22716}

Dataset loaders

In [11]:
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64,
                                               shuffle=False
                                               # shuffle_valtrain(x)
                                               , num_workers=6)
                for x in ['train', 'test']}

In [ ]:
dataset_valid = torch.utils.data.DataLoader(dsets['test'], batch_size=5, shuffle=True, num_workers=6)

In [ ]:
len(dataset_valid)

In [ ]:
dataset_valid

In [ ]:
count = 0
for data in dataset_valid:
    if count == 0:
        inputs_try,labels_try = data
    else:
        break
    count += 1

In [ ]:
labels_try

In [ ]:
inputs_try.shape

In [ ]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs_try)

imshow(out, title=[dset_classes[x] for x in labels_try])

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dset_loaders['train']))

n_images = 8

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

In [ ]:
# Get a batch of validation data
inputs, classes = next(iter(dset_loaders['test']))

n_images = 8

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

# Creating VGG Model

In [12]:
model_vgg = models.vgg16(pretrained=True)

In [ ]:
inputs_try , labels_try = var_cgpu(inputs_try,use_gpu),var_cgpu(labels_try,use_gpu)

if use_gpu:
    model_vgg = model_vgg.cuda()

In [ ]:
outputs_try = model_vgg(inputs_try)

In [ ]:
outputs_try

In [ ]:
outputs_try.shape

### Modifying the last layer and setting the gradient false to all layers

In [78]:
print(model_vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [13]:
for param in model_vgg.features.parameters():
    param.requires_grad = False
model_vgg.classifier._modules['6'] = nn.Linear(4096, 101)

In [ ]:
print(model_vgg.classifier)

In [14]:
if use_gpu:
    model_vgg = model_vgg.cuda()

## Calculating preconvoluted features

In [8]:
def preconvfeat(dataset):
    conv_features = []
    labels_list = []
    count = 0
    for data in dataset:
        count += 1
        print(count,"/",len(dataset),end='\r')
        
        inputs,labels = data
        if use_gpu:
            inputs , labels = Variable(inputs.cuda()),Variable(labels.cuda())
        else:
            inputs , labels = Variable(inputs),Variable(labels)
        
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list)

In [15]:
%%time
conv_feat_train,labels_train = preconvfeat(dset_loaders['train'])

/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 283
  " Skipping tag %s" % (size, len(data), tag))
/home/foodlovers/miniconda3/lib/python3.7/site-packages/PIL/TiffImagePlu

CPU times: user 7min 45s, sys: 4min 5s, total: 11min 51s
Wall time: 11min 58s


In [16]:
%%time
conv_feat_val,labels_val = preconvfeat(dset_loaders['test'])

CPU times: user 2min 35s, sys: 1min 22s, total: 3min 58s
Wall time: 3min 59s


### Save extracted features

In [17]:
save_array('/home/foodlovers/vgg16/conv_feat_train.bc',conv_feat_train)
save_array('/home/foodlovers/vgg16/labels_train.bc',labels_train)
save_array('/home/foodlovers/vgg16/conv_feat_val.bc',conv_feat_val)
save_array('/home/foodlovers/vgg16/labels_val.bc',labels_val)

In [18]:
conv_feat_train.shape

(67988, 512, 7, 7)

### Load extracted features

In [ ]:
conv_feat_train = load_array('/home/foodlovers/vgg16/conv_feat_train.bc')
labels_train = load_array('/home/foodlovers/vgg16/labels_train.bc')
conv_feat_val = load_array('/home/foodlovers/vgg16/conv_feat_val.bc')
labels_val = load_array('/home/foodlovers/vgg16/labels_val.bc')

## Training fully connected module

### Creating loss function and optimizer

In [19]:
for param in model_vgg.classifier.parameters():
    param.requires_grad = True

In [20]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer_vgg = torch.optim.SGD(model_vgg.classifier.parameters(),lr = lr)

In [22]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_vgg, step_size=5, gamma=0.1)

### Creating Data generator

In [9]:
def data_gen(conv_feat,labels,batch_size=64,shuffle=True):
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(labels))
        conv_feat = conv_feat[index]
        labels = labels[index]
    for idx in range(0,len(labels),batch_size):
        yield(conv_feat[idx:idx+batch_size],labels[idx:idx+batch_size],int(len(labels) / batch_size) + (len(labels) % batch_size > 0))

### Training the model

In [ ]:
def train_model_2(model, criterion,
                 train_data = None, train_labels = None,
                 test_data = None, test_labels = None,
                  optimizer = None,
                 epochs = 1,train = True, validate = False,
                shuffle = True) :
    
    if train == True :
        loss_history = []
        acc_history = []
        val_loss_history = []
        val_acc_history = []
        
    for epoch in range(epochs) :
        
        if train == True :
            #=========================TRAINING=================================#
            start_time_epoch = time.time()
            
            # scheduler.step()
            
            model.train()
    
            print("Epoch:", epoch,"/",epochs-1,"===============================================")
        
            running_loss = 0.0
            running_corrects = 0.0
            
            batches = data_gen(conv_feat=train_data,labels=train_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
        
                inputs,classes,batch_num = data

                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                inputs = inputs.view(inputs.size(0), -1)
                    
                # calulate outputs and losses
                outputs = model(inputs)
                loss = criterion(outputs,classes)       

                # autograd
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # statistics
                batch_loss = loss.data.item()
                _,preds = torch.max(outputs.data,1)
                batch_corrects = torch.sum(preds == classes.data)
                
                running_loss += batch_loss
                running_corrects += batch_corrects

                print('Batch {:d}/{:d} - Loss: {:.4f} Acc: {:.4f} - Time : {:.2f}s'.format(i+1,batch_num,
                             batch_loss/len(classes), float(batch_corrects)/len(classes), time.time() - start_time),end='\r')

            epoch_loss = running_loss / len(train_labels)
            epoch_acc = running_corrects.data.item() / len(train_labels)
            #
            
            loss_history.append(epoch_loss)
            acc_history.append(epoch_acc)
            
            print('Epoch {:d} completed in {:.2f} seconds ! Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch , time.time() - start_time_epoch, epoch_loss, epoch_acc))
            
        if validate == True :
            #=========================VALIDATING=================================#
            
            model.eval()
            
            val_loss = 0.0
            val_corrects = 0.0
            
            batches = data_gen(conv_feat=test_data,labels=test_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
                
                inputs,classes,batch_num = data
                
                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()
                    
                inputs = inputs.view(inputs.size(0), -1)

                outputs = model(inputs)
                loss = criterion(outputs,classes)        
                _,preds = torch.max(outputs.data,1)

                # statistics

                val_loss += loss.data.item()
                val_corrects += torch.sum(preds == classes.data)
                
                print('Validating batch {:d}/{:d} - {:.2f}s ...'.format(i+1,batch_num
                                                                , time.time() - start_time), end="\r")

            val_epoch_loss = val_loss / len(test_labels)
            val_epoch_acc = val_corrects.data.item() / len(test_labels)
            # 

            print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                             val_epoch_loss,val_epoch_acc))
            
            if train == False :
                return
            else :
                val_loss_history.append(val_epoch_loss)
                val_acc_history.append(val_epoch_acc)
    
    if train == False :
        return 'On fait rien!'
    elif validate == False :
        return loss_history, acc_history
    else :
        return loss_history, acc_history,val_loss_history,val_acc_history

In [24]:
history = train_model_2(model=model_vgg.classifier,criterion=criterion,
              train_data = conv_feat_train, train_labels = labels_train,
                 test_data = conv_feat_val, test_labels = labels_val,
                  optimizer = optimizer_vgg,
                 epochs = 20,train = True, validate = True,
                shuffle = True)

Epoch: 0 / 19 ===============================================
Epoch 0 completed in 50.84 seconds ! Loss: 0.0491 Acc: 0.2743
Val Loss: 0.0376 Val Acc: 0.4275 ...
Epoch: 1 / 19 ===============================================
Epoch 1 completed in 50.96 seconds ! Loss: 0.0408 Acc: 0.3769
Val Loss: 0.0355 Val Acc: 0.4562 ...
Epoch: 2 / 19 ===============================================
Epoch 2 completed in 50.71 seconds ! Loss: 0.0373 Acc: 0.4236
Val Loss: 0.0340 Val Acc: 0.4776 ...
Epoch: 3 / 19 ===============================================
Epoch 3 completed in 50.74 seconds ! Loss: 0.0344 Acc: 0.4588
Val Loss: 0.0331 Val Acc: 0.4961 ...
Epoch: 4 / 19 ===============================================
Epoch 4 completed in 50.77 seconds ! Loss: 0.0316 Acc: 0.4989
Val Loss: 0.0326 Val Acc: 0.5034 ...
Epoch: 5 / 19 ===============================================
Epoch 5 completed in 50.88 seconds ! Loss: 0.0281 Acc: 0.5491
Val Loss: 0.0322 Val Acc: 0.5088 ...
Epoch: 6 / 19 ====================

KeyboardInterrupt: 

In [25]:
lr = 0.01
optimizer_vgg = torch.optim.SGD(model_vgg.classifier.parameters(),lr = lr)

In [ ]:
train_model_2(model=model_vgg.classifier,criterion=criterion,
              train_data = conv_feat_train, train_labels = labels_train,
                 test_data = conv_feat_val, test_labels = labels_val,
                  optimizer = optimizer_vgg,
                 epochs = 5,train = True, validate = True,
                shuffle = True)

Epoch: 0 / 4 ===============================================
Epoch 0 completed in 50.82 seconds ! Loss: 0.0273 Acc: 0.5589
Val Loss: 0.0322 Val Acc: 0.5075 ...
Epoch: 1 / 4 ===============================================


Validation

In [35]:
train_model_2(model=model_vgg.classifier,criterion=criterion,
              train_data = conv_feat_train, train_labels = labels_train,
                 test_data = conv_feat_val, test_labels = labels_val,
                  optimizer = optimizer_vgg,
                 epochs = 10,train = False, validate = True,
                shuffle = True)

Val Loss: 0.0333 Val Acc: 0.5242 ...


Plot loss & accuracy history

In [ ]:
plt.plot(losses_hist)
plt.title("Loss history")

In [ ]:
plt.plot(acc_hist)
plt.title("Training accuracy history")

### Analyse the results

#### Confusion matrix

In [34]:
import sklearn.metrics

num_classes = len(dsets['train'].classes)
confu_matrix = np.zeros((num_classes,num_classes))

model_vgg.eval()

for i, data in enumerate(dset_loaders['test'], 0):
    start_time = time.time()
                
    # get the inputs
    inputs, classes = data

    if use_gpu:
        inputs , classes = inputs.cuda(), classes.cuda()

    outputs = model_vgg(inputs)

    _,preds = torch.max(outputs.data,1)
    
    confu_matrix += sklearn.metrics.confusion_matrix(classes.cpu(), preds.cpu(),labels=range(num_classes))
    
    print(i+1,"/",len(dset_loaders['test']),"batches processed",end="\r")

#### Accuracy of each class

In [40]:
acc_dict = {}
acc = []

for i in range(len(dsets['train'].classes)) :
    acc_dict[dsets['train'].classes[i]] = float(confu_matrix[i][i])/np.sum(confu_matrix[i])
    acc.append(float(confu_matrix[i][i])/np.sum(confu_matrix[i]))
    
for key, value in sorted(acc_dict.items(), key=lambda kv: kv[1],reverse = True):
    print("%s: %s" % (key, value))

deviled_eggs: 0.8558951965065502
guacamole: 0.8114035087719298
mussels: 0.8076923076923077
spaghetti_carbonara: 0.7844036697247706
spaghetti_bolognese: 0.7605633802816901
pulled_pork_sandwich: 0.7410714285714286
strawberry_shortcake: 0.7174887892376681
chicken_wings: 0.7168949771689498
caesar_salad: 0.7136563876651982
macarons: 0.696969696969697
eggs_benedict: 0.6854460093896714
chocolate_cake: 0.6853448275862069
waffles: 0.6680851063829787
onion_rings: 0.663594470046083
prime_rib: 0.6545454545454545
lasagna: 0.6523605150214592
miso_soup: 0.6519823788546255
caprese_salad: 0.6454545454545455
creme_brulee: 0.64
panna_cotta: 0.64
grilled_cheese_sandwich: 0.6375545851528385
clam_chowder: 0.6339285714285714
baklava: 0.6327433628318584
paella: 0.631578947368421
pancakes: 0.6239316239316239
pizza: 0.6127659574468085
red_velvet_cake: 0.6123348017621145
fried_rice: 0.611353711790393
bruschetta: 0.6103896103896104
seaweed_salad: 0.6073059360730594
beet_salad: 0.6071428571428571
cup_cakes: 0.6052

### Try weighted number of training examples

In [41]:
acc = [float(round(100*x))/100 for x in acc]

In [73]:
weights = nn.Softmax()(1. / torch.tensor(acc, dtype=torch.float).clone().detach())
train_labels = [item[1] for item in dsets['train'].imgs]
sample_weights = weights[train_labels]

weighted_sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights, dset_sizes['train'])

sampler={'train': weighted_sampler, 'test': None}

/home/foodlovers/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [74]:
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64,
                                                num_workers=6, sampler = sampler[x])
                for x in ['train', 'test']}

In [75]:
count = np.zeros(101)

for data in dset_loaders['train']:
    inputs_try,labels_try = data
    break

In [76]:
labels_try

tensor([ 58,  77,  55,  55,  55,  55,  55,  55,  83,  23,  55,  55,  55,  55,
         55, 100,  55,  55,  55,  55,  93,  55,  55,  47,  55,  55,  55,  55,
         55,  55,  55,  84,  55,  55,  55,  55,  55,  55,  55,  16,  55,  55,
         55,  55,  55,   4,   3,  55,  46,  55,  90,  55,  82,  55,  55,  55,
         55,  63,  32,  55,  24,  55,  55,  53])

In [79]:
model_vgg = models.vgg16(pretrained=True)

for param in model_vgg.features.parameters():
    param.requires_grad = False
model_vgg.classifier._modules['6'] = nn.Linear(4096, 101)

if use_gpu:
    model_vgg = model_vgg.cuda()

In [80]:
%%time
conv_feat_train,labels_train = preconvfeat(dset_loaders['train'])
conv_feat_val,labels_val = preconvfeat(dset_loaders['test'])

CPU times: user 10min 9s, sys: 5min 23s, total: 15min 32s
Wall time: 15min 32s


In [81]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer_vgg = torch.optim.SGD(model_vgg.classifier.parameters(),lr = lr)

In [82]:
history = train_model_2(model=model_vgg.classifier,criterion=criterion,
              train_data = conv_feat_train, train_labels = labels_train,
                 test_data = conv_feat_val, test_labels = labels_val,
                  optimizer = optimizer_vgg,
                 epochs = 20,train = True, validate = True,
                shuffle = True)

Epoch: 0 / 19 ===============================================
Epoch 0 completed in 50.67 seconds ! Loss: 0.0245 Acc: 0.7121
Val Loss: 0.0558 Val Acc: 0.1122 ...
Epoch: 1 / 19 ===============================================
Epoch 1 completed in 50.48 seconds ! Loss: 0.0190 Acc: 0.7352
Val Loss: 0.0477 Val Acc: 0.2407 ...
Epoch: 2 / 19 ===============================================
Epoch 2 completed in 50.84 seconds ! Loss: 0.0163 Acc: 0.7599
Val Loss: 0.0462 Val Acc: 0.2830 ...
Epoch: 3 / 19 ===============================================
Epoch 3 completed in 51.87 seconds ! Loss: 0.0143 Acc: 0.7837
Val Loss: 0.0455 Val Acc: 0.3050 ...
Epoch: 4 / 19 ===============================================
Epoch 4 completed in 50.37 seconds ! Loss: 0.0127 Acc: 0.8020
Val Loss: 0.0439 Val Acc: 0.3382 ...
Epoch: 5 / 19 ===============================================
Epoch 5 completed in 49.99 seconds ! Loss: 0.0113 Acc: 0.8206
Val Loss: 0.0427 Val Acc: 0.3616 ...
Epoch: 6 / 19 ====================

KeyboardInterrupt: 

In [83]:
num_classes = len(dsets['train'].classes)
confu_matrix = np.zeros((num_classes,num_classes))

model_vgg.eval()

for i, data in enumerate(dset_loaders['test'], 0):
    start_time = time.time()
                
    # get the inputs
    inputs, classes = data

    if use_gpu:
        inputs , classes = inputs.cuda(), classes.cuda()

    outputs = model_vgg(inputs)

    _,preds = torch.max(outputs.data,1)
    
    confu_matrix += sklearn.metrics.confusion_matrix(classes.cpu(), preds.cpu(),labels=range(num_classes))
    
    print(i+1,"/",len(dset_loaders['test']),"batches processed",end="\r")
    
acc_dict2 = {}

for i in range(len(dsets['train'].classes)) :
    acc_dict2[dsets['train'].classes[i]] = float(confu_matrix[i][i])/np.sum(confu_matrix[i])
    
for key, value in sorted(acc_dict2.items(), key=lambda kv: kv[1],reverse = True):
    print("%s: %s" % (key, value))

guacamole: 0.7543859649122807
deviled_eggs: 0.6768558951965066
spaghetti_carbonara: 0.6697247706422018
mussels: 0.6196581196581197
hot_dog: 0.6025641025641025
strawberry_shortcake: 0.5874439461883408
waffles: 0.5787234042553191
caesar_salad: 0.5726872246696035
spaghetti_bolognese: 0.5539906103286385
fried_rice: 0.5283842794759825
macarons: 0.5281385281385281
pulled_pork_sandwich: 0.5223214285714286
tiramisu: 0.5172413793103449
french_fries: 0.5047169811320755
onion_rings: 0.5023041474654378
beet_salad: 0.48214285714285715
chocolate_mousse: 0.48214285714285715
chicken_wings: 0.4794520547945205
chicken_quesadilla: 0.4727272727272727
donuts: 0.4698275862068966
pad_thai: 0.46956521739130436
creme_brulee: 0.4666666666666667
ravioli: 0.4661016949152542
eggs_benedict: 0.4647887323943662
pho: 0.463302752293578
miso_soup: 0.46255506607929514
pizza: 0.4553191489361702
cup_cakes: 0.4473684210526316
pancakes: 0.4444444444444444
edamame: 0.43722943722943725
samosa: 0.4372093023255814
paella: 0.4342